###### Import Dependencies & Chrome Driver

In [11]:
# import dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time

In [12]:
# choose path to driver
executable_path = {'executable_path': 'C:/bin/chromedriver'} # probably repoint this to repo for HW
browser = Browser(
    'chrome', 
    **executable_path, 
    headless = False
)

###### Scraping Web Text

In [13]:
# set url to NASA news
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [14]:
# create BeautifulSoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# retrieve most recent news title, paragraph
news_title = soup.find('div', class_ = "content_title").text.strip()
news_date = soup.find('div', class_ = "list_date").text.strip()
news_p = soup.find('div', class_ = 'article_teaser_body').text.strip()

In [16]:
# return article information
print(f'Artice Title: {news_title}, {news_date}')
print(f'Artice Content: {news_p}')

Artice Title: Mars Now, June  3, 2021
Artice Content: The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.


###### Scraping Web Image

In [17]:
# browse to featured mars images
image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(image_url)